In [ ]:
#%pip install azure-identity
#%pip install azure-cosmos
# %pip install openai

In [ ]:
import datetime
import os
import base64
import json
import sys
import requests
from openai import AzureOpenAI

In [ ]:
# Imposta i parametri per Azure OpenAI

AZURE_AI_FOUNDRY_PROJECT_ENDPOINT = "https://itafoundry.services.ai.azure.com/api/projects/firstProject"
API_VERSION = "2025-05-15-preview"
model = "gpt-4.1"

###REFRESH TOKEN
#az account get-access-token --resource https://itafoundry.services.ai.azure.com/api/projects/firstProject --query accessToken --output tsv
AGENT_TOKEN = "xxxxxxxxxxxx"  # Sostituisci con il tuo token di accesso


In [ ]:
#Agent Instructions
agentInstructions = "You are a customer support chatbot. Use the tools provided and your knowledge base to best respond to customer " \
"queries about Microsoft Learn portal."

#question
Question = "Quali sono i modelli Azure OpenAI in scadenza e con cosa posso sostituirli? " \
"Fornisci dettagli per ogni modello e versione, evidenziando le differenze tra le versioni. " \
"Includi anche le date di scadenza e le versioni alternative consigliate."


In [ ]:
#create a Foundry agent and add MCP tool

url = f"{AZURE_AI_FOUNDRY_PROJECT_ENDPOINT}/assistants?api-version={API_VERSION}"
headers = {
    "Authorization": f"Bearer {AGENT_TOKEN}",
    "Content-Type": "application/json"
}
data = {
    "instructions": f"{agentInstructions}",
    "tools": [
        {
            "type": "mcp", #il tool è di tipo MCP
            "server_label": "microsoft_docs_search", #il nome univoco per MCP Server di MSLearn è "microsoft_docs_search"
            "server_url": "https://learn.microsoft.com/api/mcp", #l'URL del server MCP di MSLearn
            "require_approval": "never"
        }
    ],
    "name": "my-MCPLearner",
    "model": f"{model}"
}

print("Creating Foundry agent...")
response = requests.post(url, headers=headers, json=data)
print("Status code:", response.status_code)
# print("Response:", response.text)

agentID = response.json().get("id")
print("Agent ID:", agentID)

In [ ]:
#create a thread
url = f"{AZURE_AI_FOUNDRY_PROJECT_ENDPOINT}/threads?api-version={API_VERSION}"
headers = {
    "Authorization": f"Bearer {AGENT_TOKEN}",
    "Content-Type": "application/json"
}
data = {}

print("Creating a thread for the agent...")
response = requests.post(url, headers=headers, json=data)
print("Status code:", response.status_code)
# print("Response:", response.text)

threadId = response.json().get("id")
print("Thread ID:", threadId)

In [ ]:
#Add a user question to the thread
url = f"{AZURE_AI_FOUNDRY_PROJECT_ENDPOINT}/threads/{threadId}/messages?api-version={API_VERSION}"
headers = {
    "Authorization": f"Bearer {AGENT_TOKEN}",
    "Content-Type": "application/json"
}
data = {
        "role": "user",
        "content": f"{Question}"
}

print("Adding user question to the thread...")
response = requests.post(url, headers=headers, json=data)
print("Status code:", response.status_code)
# print("Response:", response.text)

In [ ]:
#create a run
url = f"{AZURE_AI_FOUNDRY_PROJECT_ENDPOINT}/threads/{threadId}/runs?api-version={API_VERSION}"
headers = {
    "Authorization": f"Bearer {AGENT_TOKEN}",
    "Content-Type": "application/json"
}
data = {
	    "assistant_id": f"{agentID}",
    	"tool_resources": {
          "mcp": [
            {
                "server_label": "microsoft_docs_search", #aggiungo al run il server MCP di MSLearn
                "headers": {
                    "Authorization": f"Bearer {AGENT_TOKEN}",
                }
            }
          ]
      },
}

print("Adding user question to the thread...")
response = requests.post(url, headers=headers, json=data)
print("Status code:", response.status_code)
# print("Response:", response.text)

runID = response.json().get("id")
print("Run ID:", runID)


In [ ]:
#retrieve status of the run
url = f"{AZURE_AI_FOUNDRY_PROJECT_ENDPOINT}/threads/{threadId}/runs/{runID}?api-version={API_VERSION}"
headers = {
    "Authorization": f"Bearer {AGENT_TOKEN}",
}

print("Retrieving status of the run...")
response = requests.get(url, headers=headers)
print("Status code:", response.status_code)
# print("Response:", response.text)

status = response.json().get("status")
print("Run Status:", status)


In [ ]:
#retrieve agent response (after run is completed)
url = f"{AZURE_AI_FOUNDRY_PROJECT_ENDPOINT}/threads/{threadId}/messages?api-version={API_VERSION}"
headers = {
    "Authorization": f"Bearer {AGENT_TOKEN}",
}

print("Retrieving agent response...")
response = requests.get(url, headers=headers)
print("Status code:", response.status_code)
# print("Response:", response.text)

# Estrai il valore del messaggio
response_json = response.json()
value = response_json["data"][0]["content"][0]["text"]["value"]

# Formatta in markdown e stampa
markdown_output = f"""\
# Domanda

{Question}

# Risposta dell'agente

{value}
"""

print(markdown_output)

Retrieving agent response...
Status code: 200
# Domanda

Quali sono i modelli Azure OpenAI in scadenza e con cosa posso sostituirli? Fornisci dettagli per ogni modello e versione, evidenziando le differenze tra le versioni. Includi anche le date di scadenza e le versioni alternative consigliate.

# Risposta dell'agente

Ecco un riepilogo aggiornato sui modelli Azure OpenAI in scadenza, incluse le date di ritiro, le versioni alternative consigliate e le principali differenze tra le versioni:

### Modelli in Scadenza e Alternative Consigliate

| Modello e Versione               | Data ritiro            | Sostituzione consigliata                 | Differenze principali        |
|----------------------------------|------------------------|------------------------------------------|-----------------------------|
| gpt-35-turbo (0301, 0613)        | 13 febbraio 2025       | gpt-35-turbo (0125), gpt-4o-mini         | Le nuove versioni offrono maggiore efficienza e capacità migliorata. |
| gpt